In [ ]:
# Import packages

import cv2
import os
import numpy as np
import random
from PIL import Image
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

#Tensorflow
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras import Sequential
from tensorflow.keras.layers import (Dense, Dropout, Input,Conv2D, Flatten,
                             MaxPooling2D,BatchNormalization)
from tensorflow.keras.utils import to_categorical
from keras.callbacks import EarlyStopping



#Torch
import torch
import torch.nn as nn
from torchvision import transforms
import torch.optim as optim
from torch.optim import lr_scheduler

#Scikit_learn
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import (roc_curve, accuracy_score, ConfusionMatrixDisplay, confusion_matrix)
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold


In [ ]:
# If using Google Colab.

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Check GPU information

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu May 30 07:09:24 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   40C    P8              11W /  72W |      1MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
# Check the available RAM

from psutil import virtual_memory

ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 67.4 gigabytes of available RAM

You are using a high-RAM runtime!


In [ ]:
tf.random.set_seed(0)
random.seed(0)

In [ ]:
def load_galaxy_images(data_dir, categories, target_size):
        
        """
        Loads, resizes, and processes all JPG images from the specified directory.

        Parameters:
        data_dir (str): The directory containing the JPG images to be processed.
        target_size (tuple): The target size for resizing the images, specified as (width, height).

        Returns:
        list: A list of PIL Image objects, each representing a resized and processed image.

        The function performs the following steps:
        1. Lists all JPG image files in the specified directory.
        2. Reads each image using OpenCV.
        3. Resizes each image to the specified target size.
        4. Scales the pixel values and converts the image to a format compatible with PIL.
        5. Converts each resized image to a PIL Image object.
        6. Appends each PIL Image object to a list.
        7. Returns the list of PIL Image objects.
        """

        all_images = []

        file_path = [os.path.join(data_dir, filename) for filename in os.listdir(data_dir) if filename.endswith('.jpg')]

        for img in file_path:
            image = cv2.imread(img)
            resized_images=cv2.resize(image, target_size)
            resized_images = (resized_images * 255).astype(np.uint8)
            pil_images = Image.fromarray(resized_images)
            all_images.append(pil_images)

        return all_images

In [ ]:
g_im = '/path/to/your/direcotry/galaxy'
ng_im = '/path/to/your/direcotry/non_galaxy'

categories = ['g', 'ng']

g_img = load_galaxy_images(g_im, categories=categories[0], target_size=(200,200))
ng_img = load_galaxy_images(ng_im, categories=categories[1], target_size=(200,200))

all_data = g_img + ng_img
np.shape(all_data)

(1325, 200, 200, 3)

In [ ]:
input_size = 200

# transforms for training data
train_transform = transforms.Compose([transforms.CenterCrop(input_size),
                                      transforms.RandomRotation(90),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.RandomVerticalFlip(),
                                      transforms.RandomResizedCrop(input_size, scale=(0.8, 1.0), ratio=(0.99, 1.01)),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                                      ])


# transforms for test data
test_transform = transforms.Compose([transforms.CenterCrop(input_size),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                                      ])

In [ ]:
# Labels

galaxies_labels = np.zeros(len(g_img))
nongalaxy_labels = np.ones(len(ng_img))

all_labels = np.concatenate([galaxies_labels, nongalaxy_labels])
len(all_labels)

1325

### Classic CNN

In [ ]:
y_test_list=[]
models=[]
Y_pred=[]
accs=[]
cons=[]
test_indx=[]

b_size = 64
e_num = 30

for i in range (10):
    X_train, X_test, y_train, y_test, train_indices, test_indices = train_test_split(all_data, all_labels, np.arange(len(all_labels)), test_size=0.25, shuffle=True, random_state=None)
    y_test_list.append(y_test)
    test_indx.append(test_indices)

    y_train_encoded = to_categorical(y_train, num_classes=2)
    class_weights = {0: len(all_data) / (2*len(g_img)), 1: len(all_data) / (2*len(ng_img))}

    # Transformer for training data
    transformed_X_train=[]
    for i in range(len(X_train)):
      transformed_train_images = train_transform(X_train[i])
      new_image = np.transpose(transformed_train_images, (1, 2, 0))
      transformed_X_train.append(new_image)

    # Transformer for testing data
    transformed_X_test=[]
    for j in range(len(X_test)):
      transformed_test_images = test_transform(X_test[j])
      new_images = np.transpose(transformed_test_images, (1, 2, 0))
      transformed_X_test.append(new_images)

    # input
    x = Input(shape=(200,200,3))

    #hidden layers
    c0 = Conv2D(256, kernel_size=(3,3), strides=(1,1), padding="same")(x)
    b0 = BatchNormalization()(c0)
    m0 = MaxPooling2D(pool_size=(2, 2))(b0)
    d0 = Dropout(0.1)(m0)

    c1 = Conv2D(128, kernel_size=(3,3), strides=(1,1), padding="same")(m0)
    b1 = BatchNormalization()(c1)
    m1 = MaxPooling2D(pool_size=(2, 2))(b1)
    d1 = Dropout(0.1)(m1)

    c2 = Conv2D(64, kernel_size=(3,3), strides=(1,1), padding="same")(m1)
    b2 = BatchNormalization()(c2)
    m2 = MaxPooling2D(pool_size=(2, 2))(b2)
    d2 = Dropout(0.1)(m2)

    f = Flatten()(m2)

    # output layers
    de0 = Dense(64, activation='relu')(f)
    de1 = Dense(32, activation='relu')(de0)
    de2 = Dense(2, activation='softmax')(de1)

    model = Model(inputs=x, outputs=de2, name="cnn_transformer_galaxy_nonegalaxy")
    loss = tf.keras.losses.CategoricalCrossentropy(from_logits=False)
    optimizer = tf.keras.optimizers.Adam()
    model.compile(loss=loss, optimizer=optimizer, metrics=['accuracy'])

    # Callback Functions
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)

    history = model.fit(
    np.array(transformed_X_train), y_train_encoded,
    batch_size=b_size,
    epochs=e_num,
    verbose = 1,
    class_weight=class_weights,
    callbacks=es,
    validation_split=0.1
    )
    models.append(history)

    y_pred = model.predict(np.array(transformed_X_test))
    y_pred_labels = np.argmax(y_pred, axis=1)
    Y_pred.append(y_pred_labels)

    con0 = metrics.confusion_matrix(y_test, y_pred_labels)
    cons.append(con0)

    acc = metrics.accuracy_score(y_test, y_pred_labels)
    accs.append(acc)

Epoch 1/30
14/14 [==============================] - 41s 2s/step - loss: 1.9105 - accuracy: 0.5823 - val_loss: 8.3102 - val_accuracy: 0.5300
Epoch 2/30
14/14 [==============================] - 5s 328ms/step - loss: 0.8343 - accuracy: 0.7010 - val_loss: 4.4952 - val_accuracy: 0.6900
Epoch 3/30
14/14 [==============================] - 5s 329ms/step - loss: 0.7706 - accuracy: 0.6865 - val_loss: 5.3902 - val_accuracy: 0.5400
Epoch 4/30
14/14 [==============================] - 5s 329ms/step - loss: 0.6933 - accuracy: 0.7469 - val_loss: 1.1269 - val_accuracy: 0.6900
Epoch 5/30
14/14 [==============================] - 5s 330ms/step - loss: 0.5675 - accuracy: 0.7514 - val_loss: 2.4107 - val_accuracy: 0.5500
Epoch 6/30
14/14 [==============================] - 5s 329ms/step - loss: 0.3874 - accuracy: 0.8298 - val_loss: 0.7252 - val_accuracy: 0.6900
Epoch 7/30
14/14 [==============================] - 5s 329ms/step - loss: 0.3287 - accuracy: 0.8600 - val_loss: 0.6884 - val_accuracy: 0.6900
Epoch 8/

In [ ]:
print(np.mean(accs), np.std(accs))
plt.plot(accs)

In [ ]:
# To save any output you need.

output_el_path = '/path/to/your/direcotry'
import os
pickle_el_filename = 'performances_of_galaxy_nonegalaxy.pickle'
pickle_el_filepath = os.path.join(output_el_path, pickle_el_filename)

with open(pickle_el_filepath, 'wb') as pickle_file:
    pickle.dump(accs, pickle_file)
